# Predicting Probability of Successfully Passing the Group Phase <br> of the Football World Cup with Python

## Introduction

In this project I am going to investigate the ways of predicting the outcome of the group phase of a Football World Cup 
based on the information known prior to the beginning of the World Cup. The Football World Cup is held every four years. 
Its group phase is structured such that in the beginning of the phase 32 national football teams from all over the world 
are split in 8 groups, each consisting of 4 teams. Only 2 teams from each group can pass the group and enter 
the next stage of the tournament, i.e. the “Round of 16”. Thus in total 16 teams out of 32 can successfully surmount the group phase. Therefore the tackled problem in this project represents a binary classification problem, i.e. for each team I am going to predict a label, which can take on only one of the two values, 1 (passed) or 0 (not passed) depending whether or not the team is predicted to pass the group phase.
The data, which is known prior to the beginning of the respective Football World Cup, is basically the results of football matches played between this World Cup and the previous one, as well as the results of the qualifications for the respective World Cup for the national commands from different football confederations. There are currently six FIFA (Fédération Internationale de Football Association) confederations, depending on the geographical location (https://en.wikipedia.org/wiki/FIFA). Finding a unified feature set that would equally describe the chances of a team to pass the World Cup’s group phase (irregardless of which confederation it belongs to) is in my opinion one of the biggest challenges for this project, because each confederation has its own, i.e. different from other confederations, qualification format (https://en.wikipedia.org/wiki/FIFA_World_Cup_qualification). The main idea here would be to construct a feature set for a team, that is independent of the qualification format of a particular football confederation. An example of such feature might be e.g. an average number of goals scored by a team in the time span
between two consecutive world cups. Then having such a feature set one could train a machine learning classifier and produce prediction for a desired team which is independent of qualification structure of a particular confederation. I am going to use a standard pipeline for the development of a machine learning model, which includes splitting the data into test/train subsets, perform hyperparameter optimization and training of the model on the train subset and test prediction capability on the test subset. Additionally, I have introduced a special weighting procedure in order to compute the aggregated features, which is based on a simple idea that the games that lie further in the past shall contribute lesser to the outcome of a particular World Cup.  Finally, since the posed problem is a symmetric binary classification problem, i.e. exactly the half of the teams can pass the group phase, I am going to use accuracy as the main metric. Throughout the project Python 3 is used.

## Retrieving Football Statistics Data Using Web-Scraping with BeautifulSoup

In [ ]:
# import some modules first

from bs4 import BeautifulSoup
from tabulate import tabulate
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

import requests
import re
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import itertools
import warnings; warnings.simplefilter('ignore')

%matplotlib inline

In [ ]:
# read in labels
df_labels = pd.read_csv('countries_and_labels.csv')
# countries that took part in the world cups between 1998 and 2018
countries_to_scrap = list(set(df_labels.country.tolist()))

print(tabulate(df_labels.head(), headers='keys', tablefmt='fancy_grid',
               floatfmt=".4f"))

In [ ]:
# read in references
f = open('data', 'r')
refs = f.read()
refs = refs.split('\n')
base_ref = refs[0]
urls = refs[1:]

In [ ]:
%%time

def filter_out_non_fifa_countries(tag):
    return tag.name == "div" and "Non-FIFA" not in tag.h4.string


def construct_links_to_required_countries_and_years(url, hrefs):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    
    country_teams_tags = soup.body.section.find("div", class_="row country-teams")

    fifa_and_former_countries = \
                        country_teams_tags.find_all(filter_out_non_fifa_countries)
    
    for div in fifa_and_former_countries:
        countries = div.find_all("a", href=re.compile('/country/'))

        for country in countries:
            cntr = str(country.string).strip()
            if 'Serbia & Montenegro' in cntr:
                cntr = 'Serbia & Montenegro'
            
            if cntr in countries_to_scrap:
                dummy, c, n, team = country['href'].split('/')
                
                for year in range(1998,2019):
                    href = '/' + c + '/' + n + '/' + str(year) + '/' + team
                    hrefs.append((cntr, year, href))
            
    return hrefs


hrefs = []
for url in urls:
    hrefs = construct_links_to_required_countries_and_years(url, hrefs)

In [ ]:
%%time

# define some filter functions

def select_only_match_entries(cls):
    return cls == 'win' or cls == 'draw' or cls == 'defeat' or cls == 'missing'


def home_team_filter(cls):
    return cls == "teams home " or cls == "teams home winner"


def away_team_filter(cls):
    return cls == "teams away " or cls == "teams away winner"


# define some arrays for the data to be scrapped

dates = []
teams_home = []
teams_away = []
results_regular_time = []
results_penalties = []
game_types = []


# scrap the matches data

for href in hrefs:
    url = base_ref+href[2]
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
        
    matches_tags = soup.body.section.find('div', itemtype=re.compile('SportsTeam'))\
                       .find('div', class_="row country-details").div.find('div', \
                                                              class_="tab-content")\
                       .find('div', id="matches").table.tbody \
                       .find_all(class_=select_only_match_entries)
        
    if len(matches_tags) != 0:
        for tag in matches_tags:
            # get match date
            date = tag.find('td', class_="date").string.strip()
            dates.append(date)
                
            # get home team name
            team_home = str(tag.find('td', class_=home_team_filter).a.span.string) \
                        .strip()
            teams_home.append(team_home)
                
            # get away team name
            team_away = str(tag.find('td', class_=away_team_filter).a.span.string) \
                        .strip()
            teams_away.append(team_away)
        
            # get score in regular time
            score = tag.find('td', class_="result")
            score_regular_time = score.a.find("span", \
                                        title=re.compile('Result in regular time'))
            if score_regular_time is not None:
                score_regular_time = str(score_regular_time.string).strip()
            else: 
                score_regular_time = str(score.a.string).strip()
            results_regular_time.append(score_regular_time)
                    
            # get penalty shootout score
            score_penalty_shootout = score.a.find("span", \
                                              title=re.compile('Penalty shootout'))
            if score_penalty_shootout is not None:
                score_penalty_shootout = str(score_penalty_shootout.string).strip()
            results_penalties.append(score_penalty_shootout)
                    
            # get game type
            game_type = tag.find('td', class_="event").a
            if game_type is not None:
                game_type = str(game_type.string).strip()
            else:
                game_type = str(tag.find('td', class_="event").string).strip()
            game_types.append(game_type)


## Data Aggregation and Preprocessing with Pandas

In [ ]:
# create dataframe containing the data
raw_data = pd.DataFrame({'date': dates, 'home_team': teams_home, \
                         'team_away': teams_away, 'score': results_regular_time, \
                         'penalties': results_penalties, 'game_type': game_types})

raw_data = raw_data.drop_duplicates()

raw_data['date'] = raw_data['date'].apply(lambda r: datetime.strptime(r, '%Y-%m-%d'))

# print(tabulate(raw_data.head(), headers='keys', tablefmt='fancy_grid', \
# floatfmt=".4f"))

Below I define some auxiliary functions for the feature processing and aggregation.

In [ ]:
def get_goals_for(r):
    
    goals_for = 0
    
    if r.team[:-4] == r.home_team:
        goals_for += int(r.score.split(':')[0])
        if isinstance(r.penalties, str):
            r.penalties
            goals_for += int(r.penalties.split(':')[0])
    else:
        goals_for += int(r.score.split(':')[1])
        if isinstance(r.penalties, str):
            r.penalties
            goals_for += int(r.penalties.split(':')[1])
            
    return goals_for


def get_goals_against(r):
    
    goals_against = 0
    
    if r.team[:-4] == r.home_team:
        goals_against += int(r.score.split(':')[1])
        if isinstance(r.penalties, str):
            r.penalties
            goals_against += int(r.penalties.split(':')[1])
    else:
        goals_against += int(r.score.split(':')[0])
        if isinstance(r.penalties, str):
            r.penalties
            goals_against += int(r.penalties.split(':')[0])
            
    return goals_against


def get_features(df, raw_data_subset):

    df['goals_for']     = raw_data_subset['goals_for'].mean()
    df['goals_against'] = raw_data_subset['goals_against'].mean()
    df['win_rate']      = raw_data_subset['win'].mean()
    df['draw_rate']     = raw_data_subset['draw'].mean()
    df['goals_diff']    = raw_data_subset['goals_diff'].mean()
    
    return df


def weighted_average(group, column_to_average):
    
    data = group[column_to_average]
    weights = group['weight']
    
    return (data * weights).sum() / weights.sum()
    

def get_weighted_features(df, raw_data_subset):
    
    df['goals_for']     = weighted_average(raw_data_subset,'goals_for')
    df['goals_against'] = weighted_average(raw_data_subset,'goals_against')
    df['win_rate']      = weighted_average(raw_data_subset,'win')
    df['draw_rate']     = weighted_average(raw_data_subset,'draw')
    df['goals_diff']    = weighted_average(raw_data_subset,'goals_diff')
    
    return df

In [ ]:
df_labels["team"] = df_labels.apply(lambda row: row.country + str(row.year), axis=1)
teams_all_years = df_labels["team"].tolist()

df_labels = df_labels.drop(['country', 'year'], axis=1)

In [ ]:
# create a data frame for the final aggregated features
cols = ['team']
aggregated_data = pd.DataFrame(columns=cols)

# set to True in order to use only the matches' data one year 
# before the respective world cup
last_year = True 
# if flag weight_average True, the weighted averages of features are computed 
# such that the matches that further in the pass influence the average lesser
weight_average = True 

# loop over all 160 teams and calculate features for them
for team in teams_all_years:
    year = team[-4:]
    
    # set the interval over which to average the matches' data for a 
    # paticular team and year
    time_from = None
    time_to = None
    if year == '2018':
        time_to = datetime.strptime('2018-06-13', '%Y-%m-%d')
        if last_year:
            time_from = time_to - relativedelta(years=1)
        else:
            time_from = datetime.strptime('2014-07-14', '%Y-%m-%d')
    elif year == '2014':
        time_to   = datetime.strptime('2014-06-11', '%Y-%m-%d')
        if last_year:
            time_from = time_to - relativedelta(years=1)
        else:
            time_from = datetime.strptime('2010-07-12', '%Y-%m-%d')
    elif year == '2010':
        time_to   = datetime.strptime('2010-06-11', '%Y-%m-%d')
        if last_year:
            time_from = time_to - relativedelta(years=1)
        else:
            time_from = datetime.strptime('2006-07-10', '%Y-%m-%d')
    elif year == '2006':
        time_to   = datetime.strptime('2006-06-09', '%Y-%m-%d')
        if last_year:
            time_from = time_to - relativedelta(years=1)
        else:
            time_from = datetime.strptime('2002-07-01', '%Y-%m-%d')
    elif year == '2002':
        time_to   = datetime.strptime('2002-07-01', '%Y-%m-%d')
        if last_year:
            time_from = time_to - relativedelta(years=1)
        else:
            time_from = datetime.strptime('1998-07-14', '%Y-%m-%d')
       
    # create a data frame for the features for the current team and year
    df = pd.DataFrame([team], columns=cols)
    
    # take only matches data for the current team
    raw_data_subset = raw_data[(raw_data.home_team == team[:-4]) | \
                               (raw_data.team_away == team[:-4])]
    
    # take only the matches that took place before the specific World Cup 
    # in the year 'year' but after the previous World Cup 
    raw_data_subset = raw_data_subset[(raw_data_subset.date > time_from) & \
                                      (raw_data_subset.date < time_to)]
    
    # filter out games without score
    raw_data_subset = raw_data_subset[~(raw_data_subset.score == '-:-')]
    
    # transform raw features in order to obtain some intermediate features
    raw_data_subset['team'] = team
    
    raw_data_subset['goals_for'] = raw_data_subset.apply(get_goals_for, axis=1)
    
    raw_data_subset['goals_against'] = raw_data_subset.apply(get_goals_against, \
                                                             axis=1)
    
    raw_data_subset['goals_diff'] = raw_data_subset \
                         .apply(lambda r: r.goals_for - r.goals_against, axis=1)
    
    raw_data_subset['win'] = raw_data_subset \
                           .apply(lambda r: 1 if r.goals_diff > 0 else 0, axis=1)
    
    raw_data_subset['draw'] = raw_data_subset \
                           .apply(lambda r: 1 if r.goals_diff == 0 else 0, axis=1)
    
    raw_data_subset['weight'] = raw_data_subset \
                        .apply(lambda r: 1./ int((time_to - r.date).days), axis=1)
    
    # type conversion for the purpose of calcution of mean values later on
    raw_data_subset['goals_for'] = raw_data_subset['goals_for'].astype('float')
    raw_data_subset['goals_against'] = raw_data_subset['goals_against'] \
                                       .astype('float')
    raw_data_subset['goals_diff'] = raw_data_subset['goals_diff'].astype('float')
    raw_data_subset['win'] = raw_data_subset['win'].astype('float')
    raw_data_subset['draw'] = raw_data_subset['draw'].astype('float')
    
    # aggregate the previously filtered data in order to calculate final features
    if weight_average:
        df = get_weighted_features(df, raw_data_subset)
    else:
        df = get_features(df, raw_data_subset)
    
    aggregated_data = aggregated_data.append(df, ignore_index=True)
    

In [ ]:
#print(tabulate(aggregated_data.head()[['team', 'win_rate', 'goals_for', 'goals_against', 
#'goals_diff', 'draw_rate']], headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

In [ ]:
# merge labels and features into a single dataframe
aggregated_data_labels = pd.merge(aggregated_data, df_labels, on='team', \
                                  how='inner')

## Predicting Probability of Passing the Group Phase of the Football World Cup using Scikit-Learn

### Preliminary Analysis and Visualization

Naively thinking, one could assume that the probably out of all the above features the two, namely win_rate and goals_for, would be directly proportional to the probability of passing the group phase. But in the reality the situation is more complicated. In order to demonstrate this point the cell below can be executed in order to plot the seperation boundary for these two features as obtained from the SVM model fit.

In [ ]:
TestSize = 0.2

def plot_boundaries(aggregated_data, features):

    # fit a very simple SVM model in order to obtain seperation boundaries
    data = aggregated_data[features].values
    labels = aggregated_data['passed_group_phase'].values

    # split the aggregated teams data into test and train subsets
    data_train, data_test, labels_train, labels_test = train_test_split(data, \
                                    labels, test_size=TestSize, random_state=0)
    
    # apply feature scaling
    scaler = StandardScaler()
    data_train = scaler.fit_transform(data_train)
    data_test  = scaler.transform(data_test)
    
    # create classifier
    classifier = SVC(kernel='linear',random_state=0, gamma=0.02)
    classifier.fit(data_train, labels_train)
    
    # plot boundaries
    x_min = data_train[:,0].min()
    x_max = data_train[:,0].max()
    
    y_min = data_train[:,1].min()
    y_max = data_train[:,1].max()
    
    w = classifier.coef_[0]
    a = -w[0] / w[1]

    xx = np.linspace(x_min, x_max)
    yy = a * xx - (classifier.intercept_[0]) / w[1]
    yy_min = yy.min()
    yy_max = yy.max()
    if yy_min < y_min:
        y_min = yy_min
    if yy_max > y_max:
        y_max = yy_max

    margin = 1 / np.sqrt(np.sum(classifier.coef_ ** 2))
    yy_down = yy - np.sqrt(1 + a ** 2) * margin
    if yy_down.min() < y_min:
        y_min = yy_down.min()
 
    yy_up = yy + np.sqrt(1 + a ** 2) * margin
    if yy_up.max() > y_max:
        y_max = yy_up.max()

    plt.figure(1, figsize=(4, 3))
    plt.clf()
    plt.plot(xx, yy, 'k-')
    plt.plot(xx, yy_down, 'k--')
    plt.plot(xx, yy_up, 'k--')

    plt.scatter(classifier.support_vectors_[:, 0], \
                classifier.support_vectors_[:, 1], s=80, facecolors='none', \
                zorder=10, edgecolors='k')
    plt.scatter(data_train[:,0], data_train[:,1], c=labels_train, zorder=10, \
                cmap=plt.cm.Paired, edgecolors='k')
    
    plt.axis('tight')

    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
    Z = classifier.predict(np.c_[XX.ravel(), YY.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(XX.shape)
    plt.figure(1, figsize=(4, 3))
    plt.pcolormesh(XX, YY, Z, cmap=plt.cm.Paired)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.xticks(())
    plt.yticks(())
    
    plt.xlabel("goals_for")
    plt.ylabel("win_rate")
    
    plt.show()

    # make predicion on the test set
    #labels_pred  = classifier.predict(data_test)
    #print("Accuracy: {}".format(accuracy_score(labels_test, labels_pred)))

plot_boundaries(aggregated_data_labels, ['goals_for', 'win_rate'])

### Feature Selection

Define some auxiliary functions:

In [117]:
def classification(aggregated_data, features, clsfr):
    
    data = aggregated_data[features].values
    labels = aggregated_data['passed_group_phase'].values
    
    # split the aggregated teams data into test and train subsets
    data_train, data_test, labels_train, labels_test = train_test_split(data, \
                                    labels, test_size=TestSize, random_state=0)
    
    # apply feature scaling
    scaler = StandardScaler()
    data_train = scaler.fit_transform(data_train)
    data_test  = scaler.transform(data_test)
    
    # create classifier
    classifier = clsfr

    classifier.fit(data_train, labels_train)

    # make predicion on the test set
    labels_pred  = classifier.predict(data_test)

    # evaluate the accuracy metric
    return accuracy_score(labels_test, labels_pred)


The cell below carries out a simple feature selection:

In [ ]:
# initialize some variables for the future use
features = ['goals_for', 'goals_against', 'win_rate', 'draw_rate']
features_opt_reg = []
features_opt_svc = []
score_reg_prev = 0
score_svc_prev = 0
    
# build all possible combinations of features
feature_combinations = []
for L in range(0, len(features)+1):
    for subset in itertools.combinations(features, L):
        if list(subset):
            feature_combinations.append(list(subset))

#define classifiers
classifier_logit = LogisticRegression(random_state=0)
classifier_svc = SVC(kernel='rbf',random_state=0)

# calculate the accuracy score for each combination of features 
# and chose the feature combination with the best score
for feature_combination in feature_combinations:
        
    print ("--------------------------------------------------------------")
    print ("FEATURES: {}".format(feature_combination))
        
    # fit logistic regression
    score_reg  = classification(aggregated_data_labels, feature_combination, \
                                classifier_logit)
    print ("{0:26s} {1:10.8f}".format('LOGISTIC REGRESSION SCORE:', score_reg))
        
    # fit SVC
    score_svc  = classification(aggregated_data_labels, feature_combination, \
                                classifier_svc)
    print ("{0:26s} {1:10.8f}".format('SVC SCORE:', score_svc))
        
    # update the optimal feature combination for the logistic regression model
    # if the new score is higher than the one for the previous feature combination 
    if score_reg >= score_reg_prev:
        features_opt_reg = feature_combination
        score_reg_prev = score_reg
            
    # update the optimal feature combination for the SCV model
    # if the new score is higher than the one for the previous feature combination 
    if score_svc >= score_svc_prev:
        features_opt_svc = feature_combination
        score_svc_prev = score_svc
    
# print final results
print ("\n")
print ("Optimal features for logistic regression: {}\n Score: {}". \
       format(features_opt_reg, score_reg_prev))
print ("Optimal features for SVM: {}\n Score: {}". \
       format(features_opt_svc, score_svc_prev))

By running the cell above one would be able to see that for the current problem the classification based on the logistic regression outperforms the support vector machine classification. <br>
Further, the usage of the weighted average described above allows to achieve the accuracy score that is larger than 81%! 

### Fine-tuning

Define auxiliary function to perform the fine tuning of hyperparameters:

In [ ]:
def grid_search(classifier, parameters, aggregated_data, features):
    
    data = aggregated_data[features].values
    labels = aggregated_data['passed_group_phase'].values
    
    # split the aggregated teams data into test and train subsets
    data_train, data_test, labels_train, labels_test = train_test_split(data, \
                                    labels, test_size=TestSize, random_state=0)
    
    # apply feature scaling
    scaler = StandardScaler()
    data_train = scaler.fit_transform(data_train)
    data_test  = scaler.transform(data_test)
    
    # Create an instance of the GridSearchCV class
    clf = GridSearchCV(classifier, parameters, scoring=make_scorer(accuracy_score),\
                       cv=5, n_jobs=-1)
    
    # perform the grid search on the train data
    clf.fit(data_train, labels_train)
    
    # predict based on the best fitted model labels for the test dataset
    labels_pred  = clf.predict(data_test)
    
    return clf.best_params_, accuracy_score(labels_test, labels_pred)

By carrying out the next cell one can perform the grid-search on the optimized feature set for the logistic regression:

In [ ]:
%%time

classifier = LogisticRegression(random_state=0)
parameters = {'C': list(np.linspace(0.1,10,100)), \
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
best_params_reg, score = grid_search(classifier, parameters, \
                                     aggregated_data_labels, features_opt_reg)
print ("Score: {}".format(score))
print ("Best parameters: {}".format(best_params_reg))

By carrying out the next cell one can perform the grid-search on the optimized feature set for the SVM classification:

In [ ]:
%%time

classifier = SVC(kernel='rbf',random_state=0)
parameters = {'gamma': list(np.linspace(0.1,1,19)), \
              'C': list(np.linspace(0.5,1,50)), 'degree': [1,2,3,4,5,6,7,8,9,10] }
best_params_svm, score = grid_search(classifier, parameters, \
                                     aggregated_data_labels, features_opt_svc)
print ("Score: {}".format(score))
print ("Best parameters: {}".format(best_params_svm))